In [2]:
import pandas as pd
# Считывание данных
hotels = pd.read_csv(r'D:\IDE_github\EDA\EDA_project\data\hotels.csv')
# Вывод первых нескольких строк
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [3]:
# Создание копии датасета
data = hotels.copy()
# Вывод информации о пропусках
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [4]:
# Вывод типа данных каждого столбца
data.dtypes

hotel_address                                  object
additional_number_of_scoring                    int64
review_date                                    object
average_score                                 float64
hotel_name                                     object
reviewer_nationality                           object
negative_review                                object
review_total_negative_word_counts               int64
total_number_of_reviews                         int64
positive_review                                object
review_total_positive_word_counts               int64
total_number_of_reviews_reviewer_has_given      int64
reviewer_score                                float64
tags                                           object
days_since_review                              object
lat                                           float64
lng                                           float64
dtype: object

In [5]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей) 
X = data.drop(['reviewer_score'], axis = 1)  
y = data['reviewer_score']
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)
# Учет только численных данных
X_train = X_train.select_dtypes(include=['int64', 'float64'])
X_test = X_test.select_dtypes(include=['int64', 'float64'])
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train) 
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

In [7]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14148203980552745


Далее стоит задача повышения точности прогнозов. Нужно подготовить данные

## 1. Работа со строковыми значениями

In [8]:
# Вывод столбцов, содержащих строковые значения
data_columns_obj = data.select_dtypes(include=['object']).columns.to_list()
data_columns_obj 

['hotel_address',
 'review_date',
 'hotel_name',
 'reviewer_nationality',
 'negative_review',
 'positive_review',
 'tags',
 'days_since_review']

### 1.1 Адрес отеля

In [9]:
data['hotel_address'].head(2)

0    Stratton Street Mayfair Westminster Borough Lo...
1    130 134 Southampton Row Camden London WC1B 5AF...
Name: hotel_address, dtype: object

In [10]:
data['hotel_address'].value_counts()

hotel_address
163 Marsh Wall Docklands Tower Hamlets London E14 9SJ United Kingdom              3587
372 Strand Westminster Borough London WC2R 0JJ United Kingdom                     3206
Westminster Bridge Road Lambeth London SE1 7UT United Kingdom                     3095
Scarsdale Place Kensington Kensington and Chelsea London W8 5SY United Kingdom    2688
7 Pepys Street City of London London EC3N 4AF United Kingdom                      2379
                                                                                  ... 
Via Palmanova 153 20132 Milan Italy                                                  7
40 Rue Rene Boulanger 10th arr 75010 Paris France                                    7
Via M Buonarroti 13 Fiera Milano City 20149 Milan Italy                              6
Johann Staud Stra e 32 16 Ottakring 1160 Vienna Austria                              6
13 Rue Fran ois Ory 92120 Paris France                                               5
Name: count, Length: 1493, dt

Столбцы содержат название города и страны расположения отеля, их можно излвечь с помощью регулярных выражений

In [11]:
# Импорт бибилиотеки для работы с регулрными выражениями
import re
# Регулярное выражение для поиска сочетаний слов в конце строки
regular_form = r'([A-Z][a-z]+(?: [A-Z][a-z]+)*)$'
data['city_country'] = data['hotel_address'].apply(lambda x: re.search(regular_form, x).group())

In [12]:
# Вычленияем страну
data['country'] = data['city_country'].apply(lambda x: x if x == 'United Kingdom' else x.split()[1])
# Поиск города
data['city'] = data['city_country'].apply(lambda x: 'London' if x == 'United Kingdom' else x.split()[0])
# Удаление старого признака
data = data.drop(['city_country'], axis=1)

### 1.2 Дата

In [13]:
data['review_date'].head(2)

0    2/19/2016
1    1/12/2017
Name: review_date, dtype: object

Можно использовать для извлечения дня, месяца или года

In [ ]:
# Пребобразование в формат datetime - удобно извлекать
data['review_date'] = pd.to_datetime(data['review_date'])

### 1.3 Название отеля

In [15]:
data['hotel_name'].head(2)

0                 The May Fair Hotel
1    Mercure London Bloomsbury Hotel
Name: hotel_name, dtype: object

Можно использовать как наблюдение за самым распространенным в списке отелем. Но для модели создавать численный признак из этого затратно\
Поэтому из названия предлагается вытащить принадлежность отеля к какой-либо категории

In [16]:
data['hotel_name'].value_counts()

hotel_name
Britannia International Hotel Canary Wharf           3587
Strand Palace Hotel                                  3206
Park Plaza Westminster Bridge London                 3095
Copthorne Tara Hotel London Kensington               2688
DoubleTree by Hilton Hotel London Tower of London    2379
                                                     ... 
Ibis Styles Milano Palmanova                            7
Renaissance Paris Republique Hotel Spa                  7
Hotel Wagner                                            6
Hotel Gallitzinberg                                     6
Mercure Paris Porte d Orleans                           5
Name: count, Length: 1492, dtype: int64

In [17]:
# Извлечение сетевых брендов
data['brand'] = data['hotel_name'].str.extract(r'(Hilton|Marriott|Ibis|Mercure|Renaissance|DoubleTree|Britannia|Park Plaza|Copthorne)')
# Статистика по брендам
brand_stats = data['brand'].value_counts()

In [18]:
# Создание списков слов-индикаторов для оценки уровня отеля
luxury_keywords = ['Plaza', 'Palace', 'Royal', 'Boutique']
business_keywords = ['Hilton', 'Marriott', 'DoubleTree']
budget_keywords = ['Ibis', 'Travelodge']
# Классификация
data['class'] = 'other'
data.loc[data['hotel_name'].str.contains('|'.join(luxury_keywords)), 'class'] = 'luxury'
data.loc[data['hotel_name'].str.contains('|'.join(business_keywords)), 'class'] = 'business'
data.loc[data['hotel_name'].str.contains('|'.join(budget_keywords)), 'class'] = 'budget'

In [19]:
# Наличие терминов Spa
data['has_spa'] = data['hotel_name'].str.contains('Spa').astype(int)
# Количество слов
data['word_count'] = data['hotel_name'].str.split().str.len()
# Извлечение типа заведения
hotel_types = ['Hotel', 'Resort', 'Inn', 'Suites', 'Lodge']
data['type'] = data['hotel_name'].str.extract(f'({"|".join(hotel_types)})')[0]

In [ ]:
# Удаление старого признака
data = data.drop(['hotel_name'], axis=1)

### 1.4 Национальность рецензента

In [20]:
data['reviewer_nationality'].head(2)

0     United Kingdom 
1     United Kingdom 
Name: reviewer_nationality, dtype: object

In [21]:
# Извлечение страны
data['reviewer_country'] = data['reviewer_nationality'].str.strip()
# Группировка по регионам (пример)
europe = ['United Kingdom', 'France', 'Germany', 'Spain', 'Italy']
asia = ['China', 'Japan', 'India', 'United Arab Emirates']
data['reviewer_region'] = data['reviewer_nationality'].apply(
    lambda x: 'Europe' if x in europe else ('Asia' if x in asia else 'Other')
)
# Гость из той же страны, что и отель
data['is_local'] = (data['reviewer_nationality'] == data['country']).astype(int)

In [ ]:
# Удаление старого признака
data = data.drop(['reviewer_nationality'], axis=1)

### 1.5 Негативная оценка

In [22]:
data['negative_review'].head(2)

0           Leaving 
1     poor breakfast
Name: negative_review, dtype: object

In [23]:
# Словарь часто распространенных проблем
issues = {
    'cleanliness': ['dirty', 'clean', 'smell', 'stains'],
    'service': ['staff', 'rude', 'service', 'wait'],
    'amenities': ['WiFi', 'breakfast', 'bed', 'shower']
}
# Проверка на наличие проблемы из каждой категории в отзыве
for category, keywords in issues.items():
    data[f'issue_{category}'] = data['negative_review'].str.contains('|'.join(keywords), case=False).astype(int)

In [ ]:
# Удаление старого признака
data = data.drop(['negative_review'], axis=1)

### 1.6 Положительная оценка

In [24]:
data['positive_review'].head(2)

0     Staff were amazing
1               location
Name: positive_review, dtype: object

In [25]:
# Словарь часто распространенных слов в позитивном отзыве
pos_categories = {
    'staff': ['friendly', 'helpful', 'polite', 'attentive'],
    'comfort': ['comfortable', 'cozy', 'spacious', 'quiet'],
    'cleanliness': ['clean', 'spotless', 'tidy'],
    'location': ['central', 'convenient', 'walkable'],
    'amenities': ['breakfast', 'pool', 'view', 'wifi']
}
# Проверка на наличие слов из каждой категории словаря с положительными сочетаниями в отзыве
for cat, keywords in pos_categories.items():
    data[f'pos_{cat}'] = data['positive_review'].str.contains('|'.join(keywords), case=False).astype(int)

In [ ]:
# Удаление старого признака
data = data.drop(['positive_review'], axis=1)

### 1.7 Теги

In [26]:
data['tags'].head(2)

0    [' Leisure trip ', ' Couple ', ' Studio Suite ...
1    [' Business trip ', ' Couple ', ' Standard Dou...
Name: tags, dtype: object

Из тегов можно извлечь множество уникальных и выявить самый распространенный

In [27]:
# Множество тегов
unique_tags = {
    # Разделение строки по пробелу, удаление кавычек
    t.strip().strip("'") 
    for tag_str in data['tags']
    # Удаление скобок, разделение по запятой
    for t in tag_str.strip("[]").split(",")
}
print(unique_tags)

{' Andaz Large King Room ', ' Signature Luxury King with Balcony ', ' Classic Double Room with Breakfast ', ' Four Seasons Suite ', ' FAMILY PLAN ', ' Superior King Suite ', ' Junior Suite with Spa Access 3 Adults ', ' Cozy Double or Twin Room ', ' Large Room ', ' Junior Suite with View of the Eiffel Tower ', ' Deluxe Room 2 Adults 1 Child ', ' Twin Premier Room ', ' Family Room 6 Adults ', ' Deluxe Double with Winter Garden View ', ' King Deluxe Room ', ' Deluxe Double Twin Room ', ' Grand Suite Duplex ', ' Two Bedroom Suite ', ' Executive Room Non Smoking ', ' Double Deluxe Room ', ' Family Room Free Wi Fi ', ' Superior Room Pigalle ', ' Special Offer Double or Twin Room ', ' Landmark Junior Suite ', ' La Petite Maison ', ' Standard Room 1 2 adults ', ' Ring Suite ', ' Penthouse King Executive Room with Access to Executive Lounge ', ' Queen Room Non Smoking ', ' Club Double or Twin Room ', ' Double Room with Club Lounge Access ', ' Double Room XL ', ' Superior Twin Room with View ', 

In [28]:
from collections import Counter
# Создаем словарь для подсчета тегов
tag_counts = Counter()
for tag_str in data['tags']:
    # Преобразуем строку в список
    tags = [t.strip().strip("'") for t in tag_str.strip("[]").split(",")]
    # Обновляем счетчик
    tag_counts.update(tags)

# Находим самый частый тег
most_common_tag, count = tag_counts.most_common(1)[0]

print(f"Самый популярный тег: '{most_common_tag}' (встречается {count} раз(а))")

Самый популярный тег: ' Leisure trip ' (встречается 313593 раз(а))


Также можно получить признаки:
- тип поездки
- тип гостей
- категория номера
- длительность пребывания

In [29]:
# Функция для извлечения типа поездки
def extract_trip_type(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Leisure trip', 'Business trip', 'Romantic trip']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип поездки не найден

# Применяем функцию к столбцу с тегами
data['trip_type'] = data['tags'].apply(extract_trip_type)

In [30]:
# Функция для извлечения типа гостей
def extract_guests_type(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Solo traveler', 'Couple', 'Family', 'Group']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип гостей не найден

# Применяем функцию к столбцу с тегами
data['guests_type'] = data['tags'].apply(extract_guests_type)

In [31]:
# Функция для извлечения категории номера
def extract_category_room(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Standard Double', 'Studio Suite', 'Deluxe Room']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип номера не найден

# Применяем функцию к столбцу с тегами
data['category_room'] = data['tags'].apply(extract_category_room)

In [32]:
# Функция для извлечения длительности пребывания
def extract_timeofstay(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Stayed 1 night', 'Stayed 2 nights', 'Long stay']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип времяпребывания не найден

# Применяем функцию к столбцу с тегами
data['timeofstay'] = data['tags'].apply(extract_timeofstay)

In [33]:
# Удаление старого признака
data = data.drop(['tags'], axis=1)

### 1.8 Дни с даты ревью

In [34]:
data['days_since_review'].head(2)

0    531 day
1    203 day
Name: days_since_review, dtype: object

Можно разделить на группы, насколько давно был оставлен отзыв

In [35]:
data['days_since_review'].apply(lambda x: x.split()[1]).value_counts()

days_since_review
day     329964
days     56839
Name: count, dtype: int64

Содержит только дни, значит можно без перевода из годов в дни получить дни

In [36]:
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]))
bins = [0, 30, 90, 365, float('inf')]  # Границы категорий (дни)
labels = ['<1 month', '1-3 months', '3-12 months', '>1 year']
data['review_recency'] = pd.cut(data['days_since_review'], bins=bins, labels=labels)

In [37]:
# Удаление старого признака
data = data.drop(['days_since_review'], axis=1)

## 2. Работа с пропусками в данных

In [ ]:
# Вывод общей информации о столбцах
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 38 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

In [45]:
# Получаем список столбцов с пропусками
columns_with_nulls = data.columns[data.isnull().any()].tolist()
# Выводим результат
print("Столбцы с пропущенными значениями:")
print(columns_with_nulls)

Столбцы с пропущенными значениями:
['lng', 'brand', 'type', 'trip_type', 'guests_type', 'category_room', 'timeofstay', 'review_recency']


### 2.1 Географическая широта

In [40]:
data['lat'].head(2)

0    51.507894
1    51.521009
Name: lat, dtype: float64

In [ ]:
# Заполнение медианой в пределах города
data['lat'] = data.groupby('city')['lat'].transform(lambda x: x.fillna(x.median()))

Если модель будет плохо сходится - можно попробовать заполнять в пределах региона

### 2.2 Бренд

Поскольку этот признак вводился на этапе обработки, огично заполнить то, что не категоризировалось, значением "другие"

In [49]:
data['brand'] = data['brand'].fillna('Other')

## 3. Создание новых признаков

## 4. Преборазование признаков

## 5. Отбор признаков